# Business Data Analytics and Prediction

## Group no. 4 - Housing Prices


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
from scipy.stats import norm, skew 

from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Turn off warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# Used for development stages
DEBUG_MODE = True

In [ ]:
#Eilon - used for development stage - I need to set my working directory manually
EILON_SET_WORK_DIR = True
if (EILON_SET_WORK_DIR):
# Now change the directory
    import os
    os.chdir("C:/Users/bareilon/Documents/Personal/MBA/IDC BIG DATA/Mini Semester 1/Business Data Analytics/exercises/IDC-BDA-Exercises")

### 1. Business objectives and targets

#### Business objective: 


### 2. Read, explore and prepare data

#### 2.1. Download and read the data

The dataset is taken from a Kaggle competition: <br>
https://www.kaggle.com/c/house-prices-advanced-regression-techniques

Read the csv file to a data frame. Note that the data is already split to train and test files.

In [ ]:
df = pd.read_csv("Data/train.csv")

#### 2.2. Taking a quick look at the data 
Using the functions: 
* head
* info 
* describe


1. The prediction target (y) is the SalePrice column.
2. We will handle some missing data

In [ ]:
df.head()

In [ ]:
#df.info()

In [ ]:
df_train = pd.read_csv("Data/train.csv")
train_len = df_train.shape[0]
df_test = pd.read_csv("Data/test.csv")
df_test["SalePrice"] = 0 
df_merged = df_train.append(df_test)
#df_merged.head()

In [ ]:
df_merged.describe()

In [ ]:
corr_mat = df_train[df_train.select_dtypes(exclude='object').columns]
corr_mat = corr_mat.drop('Id', axis=1)
corr_mat = corr_mat.corr()
f, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr_mat, vmax=.8, square=True)

In [ ]:
# Top 10 Correlations
n = 10 #number of variables for heatmap
cols = corr_mat.nlargest(n, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': n}, yticklabels=cols.values, xticklabels=cols.values, vmax=.8)
plt.show()

### Fill in missing values

In [ ]:
# Find all columns with missing values
def get_null_list():
    _null_list = df_merged.isna().sum()
    _null_list = _null_list[_null_list > 0].sort_values(ascending=False)
    return _null_list

null_list = get_null_list()
null_list.head(15)

### Variable: PoolQC
If the Pool Area is 0, then the Pool QC is 'NA', as described in the Data Description file

In [ ]:
idx = (df_merged.PoolArea == 0) & (df_merged.PoolQC.isna() == True)
df_merged.loc[idx, 'PoolQC'] = df_merged.loc[idx, 'PoolQC'].fillna('NA')

#### But we still have 3 outliers. Houses with pools but the qaulity is missing.

In [ ]:
df_merged[(df_merged.PoolQC.isna() == True) & (df_merged.PoolArea > 0)]

In [ ]:
df_merged.PoolQC.value_counts()

Let's take a look at the data of the houses wit the pools but no quality grading.
We'll review these features: Overall Condition, External Condition, Garage Condition, Heating Quality etc. 

In [ ]:
qc_cols = ['OverallCond','ExterCond','ExterQual','GarageCond','GarageQual','HeatingQC','FireplaceQu','KitchenQual','BsmtFinType1','BsmtFinType2','Fence','PoolQC']
df_merged[qc_cols][(df_merged.PoolQC.isna() == True) & (df_merged.PoolArea > 0)]

We will assume the quality is Typical/Average (TA), basd on the other parameters. <br>
If there was a lot of data about houses with pools, we could build a model to predict this variable based on other quality fators. 

In [ ]:
idx = (df_merged.PoolQC.isna() == True) & (df_merged.PoolArea > 0)
df_merged.loc[idx, 'PoolQC'] = df_merged.loc[idx, 'PoolQC'].fillna('TA')
df_merged.loc[idx]

In [ ]:
df_merged.PoolQC.value_counts()

In [ ]:
df_merged.PoolQC.count()

### 'MiscFeature' Column 

Let's overview the 'MiscFeature' column
According to the data description:
MiscFeature: Miscellaneous feature not covered in other categories

In [ ]:
df_merged.MiscFeature.value_counts()

We will fill in all the null values with 'NA'.

In [ ]:
idx = df_merged.MiscFeature.isna()
df_merged.loc[idx, 'MiscFeature'] = df_merged.loc[idx, 'MiscFeature'].fillna('NA')
df_merged[idx].head()

But wait, let's check of there is a house with a Miscellaneous Value greater then 0 that we wrongly labeled 'MiscFeature' as 'NA'

In [ ]:
idx = (df_merged.MiscFeature == 'NA') & (df_merged.MiscVal > 0)
df_merged[idx]

There is one house that we missslabeled. Without other info, we will set it to "Other"

In [ ]:
df_merged.loc[idx, 'MiscFeature'] = 'Othr'
df_merged[idx]

In [ ]:
df_merged.MiscFeature.value_counts()

In [ ]:
df_merged.MiscFeature.count()

### Fence Column

In [ ]:
df_merged.Fence.value_counts()

In [ ]:
idx = df_merged.Fence.isna()
df_merged.loc[idx, 'Fence'] = df_merged.loc[idx, 'Fence'].fillna('NA')
df_merged[idx].head()

In [ ]:
df_merged.Fence.value_counts()

#### Alley Column

In [ ]:
df_merged.Alley.value_counts()

In [ ]:
idx = df_merged.Alley.isna()
df_merged.loc[idx, 'Alley'] = df_merged.loc[idx, 'Alley'].fillna('NA')
df_merged[idx].head()

In [ ]:
df_merged.Alley.value_counts()

### 'FireplaceQu' Column

In [ ]:
df_merged.FireplaceQu.value_counts()

In [ ]:
col = 'FireplaceQu'
idx = df_merged[col].isna()
df_merged.loc[idx, col] = df_merged.loc[idx, col].fillna('NA')
df_merged[idx].head()

In [ ]:
df_merged.FireplaceQu.value_counts()

Let's check if we miss labeled:

In [ ]:
df_merged[(df_merged.Fireplaces > 0) & (df_merged.FireplaceQu == 'NA')]

we're good...

### Garage relaed columns

In [ ]:
# Get all the columns related to Garages 
cols = df_merged.filter(regex='Garage').columns
cols = cols.values
cols

In [ ]:
# Find indexes of homes without garages
idx = df_merged[(df_merged.GarageType.isna()) & (df_merged.GarageYrBlt.isna()) & (df_merged.GarageFinish.isna()) & \
                (df_merged.GarageCars == 0)   & (df_merged.GarageArea == 0)    & (df_merged.GarageQual.isna())   & \
                (df_merged.GarageCond.isna()) ].index

In [ ]:
# Fill in missing values
df_merged.loc[idx, 'GarageType'] = df_merged.loc[idx, 'GarageType'].fillna('NA')
df_merged.loc[idx, 'GarageFinish'] = df_merged.loc[idx, 'GarageFinish'].fillna('NA')
df_merged.loc[idx, 'GarageQual'] = df_merged.loc[idx, 'GarageQual'].fillna('NA')
df_merged.loc[idx, 'GarageCond'] = df_merged.loc[idx, 'GarageCond'].fillna('NA')
df_merged.loc[idx, 'GarageYrBlt'] = df_merged.loc[idx, 'GarageYrBlt'].fillna(0)

In [ ]:
null_list = df_merged[cols].isna().sum()
null_list[null_list > 0].sort_values(ascending=False)

We still have a few rows with missing values ralted to Garages.

In [ ]:
idx = ((df_merged.GarageCond.isna()) | (df_merged.GarageYrBlt.isna()) | (df_merged.GarageFinish.isna()) | \
                (df_merged.GarageQual.isna()) | (df_merged.GarageCond.isna()) | (df_merged.GarageYrBlt.isna()))

In [ ]:
df_merged.loc[idx, cols]

In [ ]:
# Fill in the year the house was built
df_merged.loc[idx, 'GarageYrBlt'] = df_merged.loc[idx, 'YearBuilt']
df_merged.loc[idx,cols]

Let's fill in the 'Finish' based on the most common value for 'Detached' Garages

In [ ]:
df_merged.GarageFinish[df_merged.GarageType == 'Detchd'].value_counts()

Since most 'Detached' Garages are 'Unf' (Unfinished), we will set the value to 'Unf'.

In [ ]:
df_merged.loc[idx, 'GarageFinish'] = 'Unf'

In [ ]:
df_merged.GarageArea[df_merged.GarageType == 'Detchd'].count()
df_merged.loc[idx,cols]

We'll do the same for Garage Quality

In [ ]:
df_merged.GarageQual[df_merged.GarageType == 'Detchd'].value_counts()

In [ ]:
# We will use the most common quality for Detached Garages which is 'TA'
df_merged.loc[idx, 'GarageQual'] = 'TA'

And again for Garage Finish

In [ ]:
# We will use the most common Condition for Detached Garages which is 'TA'
df_merged.loc[idx, 'GarageCond'] = 'TA'

In [ ]:
df_merged.GarageCond[df_merged.GarageType == 'Detchd'].value_counts()

We still need to handle row id 1116.
Let's try to estimate the size of the Garage, based on the year the house was built.
We assume garages back then had some stadard size, and most families owned one car.

In [ ]:
_min = 1920
_max = 1925
df_merged[(df_merged.GarageYrBlt > _min) & (df_merged.GarageYrBlt < _max)].shape[0]

In [ ]:
df_merged['GarageCars'][(df_merged.GarageYrBlt > _min) & (df_merged.GarageYrBlt < _max)].mean()

In [ ]:
idx = 1116
garage_mean_area = df_merged['GarageArea'][(df_merged.GarageYrBlt > _min) & (df_merged.GarageYrBlt < _max) & (df_merged.GarageCars == 1)].mean()
df_merged.loc[idx, 'GarageCars'] = 1.0
df_merged.loc[idx, 'GarageArea'] = round(garage_mean_area)

### LotFrontage
It seems this column is tricky. <br>
Let's examine the correlation between the 'SalePrice' and the 'LotFrontage' 

In [ ]:
df_corr = df_merged[:train_len].copy()
df_corr['SalePrice'] = df.SalePrice.astype(int)
df_corr = df_corr[df_corr.LotFrontage > 0] 
df_corr = df_corr.loc[: ,['LotFrontage','SalePrice']]
df_corr.corr()

There is a 35% correlation which is not too high, nor too low.
For now, we will fill in the Mean.

LotFrontage : Since the area of each street connected to the house property most likely have a similar area to other houses in its neighborhood , we can fill in missing values by the median LotFrontage of the neighborhood.

In [ ]:
#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
df_merged["LotFrontage"] = df_merged.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

In [ ]:
sns.distplot(df_merged.LotFrontage)

### Basement Realted Columns
BsmtExposure    82 <br>
BsmtCond        82 <br>
BsmtQual        81 <br>
BsmtFinType2    80 <br>
BsmtFinType1    79 <br>

and we also have other related columns: <br>
TotalBsmtSF <br>
BsmtUnfSF <br>
BsmtFullBath <br>
BsmtHalfBath <br>

In [ ]:
cols = df_merged.filter(regex='Bsmt').columns
cols = cols.values
cols

In [ ]:
# Fill in 0 if the TotalBsmtSF is null
idx = df_merged.TotalBsmtSF.isna()
df_merged.loc[idx, 'TotalBsmtSF'] = df_merged.loc[idx, 'TotalBsmtSF'].fillna(0.0)

In [ ]:
# Find indexes of rows with missing Basement parameters
idx = ((df_merged.BsmtQual.isna()) & (df_merged.BsmtCond.isna()) & (df_merged.BsmtExposure.isna()) & \
                (df_merged.BsmtFinType1.isna()) & (df_merged.BsmtFinType2.isna()))

In [ ]:
df_merged.loc[idx, cols].head()

In [ ]:
idx = (df_merged.TotalBsmtSF == 0.0)
str_cols = ['BsmtExposure','BsmtCond','BsmtQual','BsmtFinType1','BsmtFinType2']
df_merged.loc[idx, str_cols] = df_merged.loc[idx, str_cols].apply(lambda x: x.fillna('NA'), axis=0)

In [ ]:
idx = ((df_merged.TotalBsmtSF == 0) & ((df_merged.BsmtUnfSF.isna()) | (df_merged.BsmtFullBath.isna()) | \
                (df_merged.BsmtHalfBath.isna())))
num_cols = ['BsmtFinSF1','BsmtFinSF2','BsmtHalfBath','BsmtFullBath', 'BsmtUnfSF']
df_merged.loc[idx, num_cols] = df_merged.loc[idx, num_cols].apply(lambda x: x.fillna(0), axis=0)

In [ ]:
df_merged.loc[:,cols][df_merged.loc[:,cols].isnull().any(axis=1)]

In [ ]:
df_merged.BsmtQual.value_counts()
df_merged[df_merged.BsmtQual.isna()] = df_merged[df_merged.BsmtQual.isna()].fillna('TA')

In [ ]:
df_merged.BsmtCond.value_counts()
df_merged[df_merged.BsmtCond.isna()] = df_merged[df_merged.BsmtCond.isna()].fillna('TA')

In [ ]:
df_merged.BsmtExposure.value_counts()
df_merged[df_merged.BsmtExposure.isna()] = df_merged[df_merged.BsmtExposure.isna()].fillna('No')

In [ ]:
df_merged.BsmtFinType2.value_counts()
df_merged[df_merged.BsmtFinType2.isna()] = df_merged[df_merged.BsmtFinType2.isna()].fillna('Unf')

In [ ]:
mas_cols = ['MasVnrType', 'MasVnrArea']
df_merged[mas_cols][df_merged[mas_cols].isnull().any(axis=1)].head(10)

In [ ]:
df_merged.MasVnrType.value_counts()

In [ ]:
idx = ( (df_merged.MasVnrType.isna()) & (df_merged.MasVnrArea > 0.0) )
df_merged.loc[idx, 'MasVnrType'] = df_merged.loc[idx, 'MasVnrType'].fillna('BrkFace')

In [ ]:
idx = ( (df_merged.MasVnrType.isna()) & (df_merged.MasVnrArea.isna()) )
df_merged.loc[idx, 'MasVnrType'] = df_merged.loc[idx, 'MasVnrType'].fillna('None')
df_merged.loc[idx, 'MasVnrArea'] = df_merged.loc[idx, 'MasVnrArea'].fillna(0.0)

### Let's check what is still missing

In [ ]:
null_list = get_null_list()
null_list

MSZoning Column <br>
4 missing

In [ ]:
df_merged.MSZoning.value_counts()

'RL' is the most common value so we can assume the missing values are RL.
We can also try later to reun the model with nulls and see the differance.

In [ ]:
idx = df_merged.MSZoning.isna()
df_merged.loc[idx, 'MSZoning'] = df_merged.loc[idx, 'MSZoning'].fillna('RL')
df_merged.loc[idx]

In [ ]:
df_merged.Functional.value_counts()

Typ (Typical) is the most common value.
The data description file tells us we can assume Typical unless specified differently:
Functional: Home functionality (Assume typical unless deductions are warranted)
Hence, we will fill in Typ for the missing values.

In [ ]:
df_merged[df_merged.Functional.isna()]

In [ ]:
col = 'Functional'
df_merged[col] = df_merged[col].fillna('Typ')

In [ ]:
null_list = get_null_list()
null_list

#### Utilities Column

In [ ]:
col = 'Utilities'
df_merged[df_merged[col].isna()]

In [ ]:
df_merged[col].value_counts()

'AllPub' is the most common value (99.99 percent)
All records are "AllPub", except for one "NoSeWa" and 2 missing values. Since the house with 'NoSewa' is in the test set (no price info), this feature won't help in predicting the price. We will drop it.

In [ ]:
df_merged = df_merged.drop(col,  axis=1)

These columns: SaleType, KitchenQual, Electrical, Exterior2nd, Exterior1st
have 1 or 2 missing values so we will fill in with the most common value.

In [ ]:
null_cols = ['SaleType', 'KitchenQual', 'Electrical', 'Exterior2nd', 'Exterior1st']
for col in null_cols:
    df_merged[col] = df_merged[col].fillna(df_merged[col].mode()[0])

No more missing data:

In [ ]:
null_list = get_null_list()
null_list

In [ ]:
import seaborn as sns; sns.set()
# Sale Price and Lot Area pairplot
sns.distplot(df_merged[:train_len-100].LotArea)
ax = sns.scatterplot(x="LotArea", y="SalePrice", data=df_merged[:train_len])

In [ ]:
# df_merged.LotArea[df_merged.LotArea > 100000] = (df_merged.LotArea[df_merged.LotArea > 100000] / 10)

In [ ]:
ax = sns.scatterplot(x="GrLivArea", y="SalePrice", data=df_merged[:train_len])

In [ ]:
# Transform Nominal variables into dummies
df_merged = pd.get_dummies(df_merged)

# Feature Engineering

In [ ]:
# Changing MSSubClass into a categorical variable
df_merged['MSSubClass'] = df_merged['MSSubClass'].astype(str)

# Changing OverallCond into a categorical variable
df_merged['OverallCond'] = df_merged['OverallCond'].astype(str)

# Year and month sold are transformed into categorical features.
df_merged['YrSold'] = df_merged['YrSold'].astype(str)
df_merged['MoSold'] = df_merged['MoSold'].astype(str)

### Adding Total Living Area column

In [ ]:
cols = ['GrLivArea', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'BsmtUnfSF','LowQualFinSF', 'TotLivArea']
df_merged['TotLivArea'] = df_merged['1stFlrSF'] + df_merged['2ndFlrSF'] + df_merged['TotalBsmtSF']
df_merged[cols].head(10)

#### 2.3. Exploratory data analytics


Data exploration is required to understand what is your data, and which preparations are required on it. A common visualization is a histogram. Use histogram to understand the values distribution, because many statistical model assume normal distribution.

In [ ]:
if (not DEBUG_MODE):
    df_merged.hist(bins = 25, figsize = (20,15)) #Check the hist parameters by clicking on the Tab completion. 
    plt.show()

In [ ]:
if (not DEBUG_MODE):
    df_merged.LotArea[df_merged['LotArea'] < 5000].hist(bins = 50, figsize = (10,5)) #Check the hist parameters by clicking on the Tab completion. 
    plt.show()

In [ ]:
if (not DEBUG_MODE):
    display(df_merged.isnull().sum(),df_merged.isna().sum())

#### 2.4. Data manipulations

After we looked at the data we can prepare it to analysis. 

From understanding the histograms, and talking with experts we might decide to drop or change columns, or to split the data by rows. 

For example: 

...

An option that we will take here to handle this is:
1. Take out the few rows and put them aside for separate handling
2. Drop this predictor from data

(Note that after the data manipulation you can rerun the describe or histogram above to see changes)

Typically data scientists continue back and forth between diffrent data visualizations and manipulations, but for this exercise we will stop here. We will dive more to this on next lesson.

In [ ]:
df_merged.describe()

#### 2.5. Categorical values

Data manipulations require also handling of categorical data

#### 2.6. Transform the data to matrix of X and y, spliting to Train and Test

Let's now prepae to modeling:
1. Split between X the predictors and y the target
2. Turn from data frame to matrix
3. Split X and y to train data set and test data set, with matching indexes between X and y


The train_test_split is the first function we are using from sklearn.
Learn more about it function at: <br>
http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html 


sklearn handles numpy arrays, whereas until now we handled a dataframe.
Lets check that indeed we changed the type:

## Target Variable

SalePrice is the variable we decided to predict. So let's analyze this variable.
We plotted a graph of salePrice. Since target distribution is not normalized, we performed log-transformation. 

In [ ]:
df_corr = df_merged[:train_len].copy()
df_corr['SalePrice'] = df.SalePrice.astype(int)
sns.distplot(df_corr['SalePrice'], fit=norm)

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(df_corr['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(df_corr['SalePrice'], plot=plt)
plt.show()

df_corr['SalePriceLog'] = np.log1p(df_corr['SalePrice'])
#Check the new distribution 
sns.distplot(df_corr['SalePriceLog'] , fit=norm);
# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(df_corr['SalePriceLog'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(df_corr['SalePriceLog'], plot=plt)
plt.show()

# Prepare Data for the Model

In [ ]:
from sklearn.metrics import mean_squared_error

def calc_rmse(y_actual, y_predicted):
    rmse = np.sqrt(mean_squared_error(y_actual, y_predicted))
    return(rmse)

Split to train and test

In [ ]:
df = df_merged[:train_len].copy()
df['SalePriceLog'] = np.log1p(df['SalePrice'])
df = df.drop(['Id','SalePrice'],axis=1)
#split the data into train and test sections
train_df = df
#train_df, test_df= train_test_split (df, test_size = 0.1, random_state=7) 
# train_test_split is from sklearn. It requiers that the data will be a numpy array, that is numbers only.

In [ ]:
# Prepare Train Data
X_train = train_df.drop('SalePriceLog',axis=1).values
y_train = train_df['SalePriceLog'].values

### 3. Fit Models

We will start with the basic model of Linear Regression

### 3.1 Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

#### 3.2 Fit the selected model
Training the model, using sklearn, is typically only one command.

In [ ]:
reg.fit(X_train, y_train) 

### 4. Model evaluation

Evaluating the model can also be done in one command.

We can evalute the model that was trained on train_x by its prediction of test_x compared to test_y in one command. On next lessons we will learn more evaluation methods, as usually decision is taken by combined evaluation scores.

In [ ]:
y_reg_pred = reg.predict(X_train)
calc_rmse(y_train, y_reg_pred)

As we can see the evaluation on test was very good. Yet, model score on train is much higher. This may indicate that we might be in an overfit to the train data set.

# Second Model
### Gradient Boosting Regression


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1,
                                   max_depth=8, max_features='sqrt',
                                   min_samples_leaf=10, min_samples_split=4, 
                                   loss='huber', random_state=42)

### Train the model

In [ ]:
gbr.fit(X_train, y_train)

### Evaluate

In [ ]:
y_gbr_pred = gbr.predict(X_train)
calc_rmse(y_train, y_gbr_pred)

## Random Forrest

1000,12 -> 0.05486181834047261   100,20 -> 0.0.05248432308239742

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rnf = RandomForestRegressor(n_estimators=100, max_depth=20, n_jobs=-1)
rnf.fit(X_train, y_train)

In [ ]:
y_rnf_pred = rnf.predict(X_train)
calc_rmse(y_train, y_rnf_pred)

### Simple Stacked Models 

In [ ]:
y_average = 0.5 * y_gbr_pred + 0.3 * y_rnf_pred +  + 0.2 * y_reg_pred
calc_rmse(y_train, y_average)

y_pred = rnf.predict(X_test)
plt.figure(figsize=(10,10))
plt.subplot(1, 1, 1)
plt.scatter(y_test, y_pred, label = 'Predict')
plt.show

### 5. Predict / Deploy model

Predict on test data from Kaggle

In [ ]:
df = df_merged[train_len:].copy()
X_pred = df.drop(['Id','SalePrice'],axis=1).values
y_pred = np.exp(0.5 * gbr.predict(X_pred) + 0.3 * rnf.predict(X_pred) +  0.2 * reg.predict(X_pred))

In [ ]:
kaggle_df = pd.DataFrame()
kaggle_df['Id'] = df_merged.Id[train_len:]
kaggle_df['SalePrice'] = y_pred
kaggle_df.to_csv('Prediction.csv', index=False)

### 6. Communicate
Now that you have a good working model you need to communicate your results.

If this is a predict project, you may decide not to communicate details externaly, only your evaluation results.

You need to communicate how you got to the results, to customers on infer project, and internaly on predict project. 

In [ ]:
tmp = df_merged.iloc[:train_len].copy()
df_hb = tmp[tmp.HalfBath > 0].copy()
df_hb['HalfBath'] = df_hb['HalfBath'] - 1 
df_hb['FullBath'] = df_hb['FullBath'] + 1 
X = df_hb.drop(['Id','SalePrice'],axis=1).values
y_hb = np.exp(gbr.predict(X))

Calculate the price differance and plot the distribution

In [ ]:
diff =  y_hb - df_hb['SalePrice']

limit = 30000
plt.xlim(-limit, limit)
plt.axvline(10000, color='coral')
sns.distplot(diff ,color="teal", axlabel="Predicted Price Change")

### Feature Score (GBR)

In [ ]:
feature_score = gbr.feature_importances_
feature_names = df_merged.drop(['Id','SalePrice'], axis=1).columns
df_feature_score = pd.DataFrame(data=feature_names, columns=["Feature"])
df_feature_score["score"]= feature_score
df_feature_score = df_feature_score.sort_values(by=['score'], ascending=False)
df_feature_score.head()

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot(y="Feature", x="score", data=df_feature_score.head(26))

## Grid Search optimization for Random Forest model

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_n_estimators_range = [10, 100, 500]
param_max_depth_range =  [7, 12, 15, 20] 

param_grid = [{'n_estimators': param_n_estimators_range, 'max_depth':param_max_depth_range},
              {'n_estimators': param_n_estimators_range, 'max_depth':param_max_depth_range, 'bootstrap': ['False']}]


In [ ]:
gs = GridSearchCV (estimator=RandomForestRegressor(), param_grid = param_grid, cv=10)

In [ ]:
#gs = gs.fit (X=X_train, y=y_train.ravel())

In [ ]:
print ('The best score is:', gs.best_score_, '\nThe best parameters are:', gs.best_params_)

## Grid Search optimization for GBR

In [ ]:
param_n_estimators_range = [1000, 3000]
param_max_depth_range =  [4, 8] 
param_learning_rate = [0.05, 0.1]

param_grid = [{'n_estimators': param_n_estimators_range, 'max_depth':param_max_depth_range, 'learning_rate':param_learning_rate}]
gs = GridSearchCV (estimator=GradientBoostingRegressor(), param_grid = param_grid, cv=10)
gs = gs.fit (X=X_train, y=y_train.ravel())
print ('The best score is:', gs.best_score_, '\nThe best parameters are:', gs.best_params_)